In [ ]:
#################################################
################  Import things #################
#################################################

import numpy as np
import timeit
import random
import copy
import matplotlib.pyplot as plt
from sklearn import datasets
import pickle
import glycowork


In [2]:
from glycowork.glycan_data.loader import glycan_binding as gb

gb_glycans = list(gb.columns)

In [3]:
from glycowork.glycan_data.loader import glycomics_data_loader
df = glycomics_data_loader.human_skin_O_PMC5871710 # imports as a Pandas dataframe

df.head(20)

,glycan,control_1,tumor_1,control_2,tumor_2,control_3,tumor_3,control_4,tumor_4,control_5,tumor_5,control_6,tumor_6,control_7,tumor_7,control_8,tumor_8,control_9,tumor_9
0,GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc,2.755913,0.620301,3.019806,4.148615,2.413021,0.424605,1.764977,0.744528,4.518547,0.849240,3.174455,1.543922,3.752813,1.287601,1.574444,0.627539,4.415230,0.425039
1,Gal(b1-3)[Neu5Ac(a2-6)]GalNAc,0.775234,0.178490,0.778083,0.832907,0.824018,0.410873,0.617573,2.133773,0.603573,0.544599,1.317444,1.256997,0.860332,0.340373,0.927956,0.494144,1.045950,0.590404
2,Neu5Ac(a2-3)Gal(b1-3)GalNAc,9.219494,8.436141,16.190721,11.032313,16.061363,8.133529,12.291188,10.143781,10.865172,14.353894,12.313024,11.601938,13.246380,8.352032,12.352587,9.731834,16.170263,12.693331
3,Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc,2.770825,0.944913,1.531698,2.427562,1.570204,2.066165,2.832475,2.508954,2.228299,1.757840,4.811796,3.641077,1.437487,1.470697,2.324956,2.192553,0.805130,1.100490
4,Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc,35.912676,30.058418,48.420833,43.738436,46.692847,26.728713,39.971128,32.290024,51.472292,40.669153,41.927351,37.755641,43.701528,32.565498,47.145757,38.565763,44.683562,33.532942
5,Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)...,0.664610,0.495280,1.451470,1.289237,1.331164,0.441114,0.657772,0.330536,0.654086,0.219399,1.433102,0.791533,0.722264,0.589715,1.337691,0.779090,1.734299,0.605036
6,Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc,1.069934,1.735744,1.468105,1.509499,1.128653,1.812381,0.816789,6.345353,2.179460,1.261105,1.438459,4.410718,0.669910,1.310332,1.838577,0.989791,1.329698,1.392818
7,Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]...,0.599954,1.213936,0.450138,0.469515,0.528332,2.891749,0.770085,0.853955,0.853148,4.776427,0.669361,1.323619,0.914651,1.027152,1.633415,4.333793,0.365928,3.162330
8,Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]...,6.005921,6.365952,5.623333,5.965819,5.299405,7.027606,10.523154,12.570391,4.753029,6.895145,12.325022,10.338465,5.401390,5.785955,7.054084,5.775716,11.549820,4.633692
9,Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]Ga...,3.557548,2.179078,2.341708,2.594836,2.408925,1.838783,3.403741,1.561446,1.909720,0.838742,1.389057,1.590800,2.557034,3.846191,0.765387,0.660688,1.428603,0.845144


In [4]:
from glycowork.glycan_data.loader import glycan_binding as gb

gb_glycans = list(gb.columns)

glycans_in_df = list(df['glycan'])

for glycan in glycans_in_df:
    if glycan in gb_glycans:
        print(glycan)

Gal(b1-3)[Neu5Ac(a2-6)]GalNAc
Neu5Ac(a2-3)Gal(b1-3)GalNAc
Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc
Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc
Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]GalNAc
Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc
Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)Gal(b1-3)]GalNAc


In [50]:
gb.head()

,3-Anhydro-Gal(a1-3)Gal(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal2S(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3dGal(b1-3)[Fuc(a1-4)]Glc,3dGal(b1-4)Glc,4d8dNeu5Ac(a2-3)Gal(b1-4)Glc,4dNeu5Ac(a2-3)Gal(b1-4)Glc,7dNeu5Ac(a2-3)Gal(b1-4)Glc,...,wwwSflexneri5c,wwwSflexneriO2c,wwwSflexneriO5c,wwwSisomicin,wwwSmix,wwwTobramycin,wwwTyrS,wwwpHGGs,target,protein
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AADSIPSISPTGIITPTPTQSGMVSNCNKFYDVHSNDGCSAIASSQ...,TAL6-4LysM
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFFSLVVLLALLPFGIHASALPSTELTPRVNPNLPGPNDVFVGFR...,rCnSL-proA
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AANEADYQAKLTAYQTELARVQKANADAKAAYEAAVAANNAANAAL...,AntigenI/IIA3VP1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AASKLGVPQPAQRDQVNCQLYAVQPNDNCIDISSKNNITYAQLLSW...,TAL6-6LysM
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACNNEWEDEQYEQYISFKSPIPAGGEGVTDIYVRYKEDGKVTYRLP...,SP15308A-bot-339-19-339


In [76]:
for i, prot in enumerate(gb['protein']):
    if prot == "ConA":
        print(i)
        print(prot)
        print(gb['target'][i])

    if prot == "WGA":
        print(i)
        print(prot)
        print(gb['target'][i])

v = gb[gb['protein'] == "ConA"]['target'].iloc[0]
v = gb[gb['protein'] == "WGA"]['target'].iloc[0]
print(v)

print(gb.iloc[407]['protein'])


34
ConA
ADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGKVGTAHIIYNSVDKRLSAVVSYPNADSATVSYDVDLDNVLPEWVRVGLSASTGLYKETNTILSWSFTSKLKSNSTHETNALHFMFNQFSKDQKDLILQGDATTGTDGNLELTRVSSNGSPQGSSVGRALFYAPVHIWESSAVVASFEATFTFLIKSPDSHPADGIAFFISNIDSSIPSGSTGRLLGLFPDAN
407
ConA
MAISKKSSLFLPIFTFITMFLMVVNKVSSSTHETNALHFMFNQFSKDQKDLILQGDATTGTDGNLELTRVSSNGSPQGSSVGRALFYAPVHIWESSAVVASFEATFTFLIKSPDSHPADGIAFFISNIDSSIPSGSTGRLLGLFPDANVIRNSTTIDFNAAYNADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGKVGTAHIIYNSVDKRLSAVVSYPNADSATVSYDVDLDNVLPEWVRVGLSASTGLYKETNTILSWSFTSKLKSNEIPDIATVV
1344
WGA
QRCGEQGSNMECPNNLCCSQYGYCGMGGDYCGKGCQNGACWTSKRCGSQAGGATCPNNHCCSQYGHCGFGAEYCGAGCQGGPCRADIKCGSQSGGKLCPNNLCCSQWGFCGLGSEFCGGGCQSGACSTDKPCGKDAGGRVCTNNYCCSKWGSCGIGPGYCGAGCQSGGCDAVFAGAITANSTLLAE
QRCGEQGSNMECPNNLCCSQYGYCGMGGDYCGKGCQNGACWTSKRCGSQAGGATCPNNHCCSQYGHCGFGAEYCGAGCQGGPCRADIKCGSQSGGKLCPNNLCCSQWGFCGLGSEFCGGGCQSGACSTDKPCGKDAGGRVCTNNYCCSKWGSCGIGPGYCGAGCQSGGCDAVFAGAITANSTLLAE
ConA


In [49]:
gb['target']

0       AADSIPSISPTGIITPTPTQSGMVSNCNKFYDVHSNDGCSAIASSQ...
1       AAFFSLVVLLALLPFGIHASALPSTELTPRVNPNLPGPNDVFVGFR...
2       AANEADYQAKLTAYQTELARVQKANADAKAAYEAAVAANNAANAAL...
3       AASKLGVPQPAQRDQVNCQLYAVQPNDNCIDISSKNNITYAQLLSW...
4       ACNNEWEDEQYEQYISFKSPIPAGGEGVTDIYVRYKEDGKVTYRLP...
                              ...                        
1460    YGAPAGPLIVPYNLPLPGGVVPRMLITILGTVKPNANRIALDFQRG...
1461    YIGDFRCIQLVNSNGANVSAPSISTETVEVSQGLGTYYVLDRVYLN...
1462    YKLVCYFTQWSQDRQEPGKFTPENIDPFLCSHLIYSFASIENNKVI...
1463    YRRTCSHTGKGEGWYIIRRGDNFNAVAADFCTSTNVLTEWNHISTI...
1464    YVDVGPDSVKSACIEVDIQQTFFDKTWPRPIDVSKADGIIYPQGRT...
Name: target, Length: 1465, dtype: object

In [17]:
dir(glycomics_data_loader)

['HIV_gagtransfection_N_PMID35112714',
 'HIV_gagtransfection_O_PMID35112714',
 'human_brain_GSL_PMID38343116',
 'human_brain_N_PMID38343116',
 'human_brain_O_PMID38343116',
 'human_colorectal_N_PMID26085185',
 'human_colorectal_O_PMC9254241',
 'human_colorectal_O_PMID19152289',
 'human_gastric_O_PMC4816881',
 'human_gastric_O_PMC5762837',
 'human_gastric_O_PMC7226152',
 'human_gastric_O_PMID28461410',
 'human_leukemia_N_PMID34646384',
 'human_leukemia_O_PMID34646384',
 'human_liver_O_PMC5383776',
 'human_liver_O_PMC9254241',
 'human_ovarian_O_PMC4468167',
 'human_platelets_N_PMID36952551',
 'human_platelets_O_PMID36952551',
 'human_prostate_N_PMC8010466',
 'human_prostate_O_PMC8010466',
 'human_retina_GSL_PMC5173345',
 'human_serum_bacteremia_N_PMID33535571',
 'human_skin_O_PMC5871710',
 'human_skin_O_PMC5871710_BCC',
 'human_skin_O_PMC5871710_SCC',
 'mouse_brain_GSL_PMID39375371',
 'time_series_HMO_PMID22649065',
 'time_series_N_PMID32149347',
 'time_series_O_PMID32149347']

In [ ]:
# need to previously `run pip3 install esm`
import esm
from glycowork.ml.inference import get_esm1b_representations, get_lectin_preds
from glycowork.ml.models import prep_model


model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S() # will need to download if the first time

In [46]:
# see examples from https://colab.research.google.com/github/BojarLab/glycowork/blob/master/05_examples.ipynb#scrollTo=honest-giant

prot_seq = "AADSIPSISPTGIITPTPTQSGMVSNCNKFYDVHSNDGCSAIASSQ"
rep = get_esm1b_representations([prot_seq], model, alphabet)

leor = prep_model('LectinOracle', 1, trained = True)

predictions = get_lectin_preds(prot_seq, glycans_in_df, leor, rep)
print(predictions)
#print(predictions['pred'])

                                                motif      pred
9   Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]Ga... -0.289560
11  Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Neu5Ac(a2-3)... -0.258926
7   Neu5Ac(a2-3)Gal(b1-3)[Gal(b1-4)GlcNAc6S(b1-6)]... -0.238169
1                       Gal(b1-3)[Neu5Ac(a2-6)]GalNAc -0.214159
4           Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GalNAc -0.210999
0                     GalOS(b1-3)[Neu5Ac(a2-6)]GalNAc -0.209233
10  Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]Ga... -0.137875
12  Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Neu5Ac(a2-... -0.127279
2                         Neu5Ac(a2-3)Gal(b1-3)GalNAc -0.111119
5   Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-8)Neu5Ac(a2-6)... -0.109295
8   Neu5Ac(a2-3)Gal(b1-4)GlcNAc6S(b1-6)[Gal(b1-3)]... -0.049835
6     Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc -0.046738
3              Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc  0.010438
